# Topic Visualization

In [8]:
from pprint import pprint
import pandas as pd
import squarify 
import plotly.express as px
import warnings
import numpy as np
from functools import reduce
import os
import datetime 
import decimal
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
%matplotlib inline  
from gensim.parsing.preprocessing import STOPWORDS
from gensim.utils import simple_preprocess
from gensim.models import TfidfModel, LsiModel
from gensim.models.ldamodel import LdaModel
from gensim import corpora
from gensim import matutils
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from collections import defaultdict
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
import lda

## Load Data

In [6]:
## Import data
df = pd.read_feather('../data/Data_Clean')
df = df.sort_values("Date",ascending="False")[["Date","Value","S_alter","Language"]]

## Topic modeling and visualization

Now that the text of all participant, we can proceed to light preprocessing before applying Latent Dirichlet Allocation.

In [9]:
# Import german stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = list(stopwords.words('german'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dominik/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
def tokenize(text):
    return [token for token in simple_preprocess(text)if token not in stopwords]

In [11]:
feedback = df.Value
documents = feedback.tolist()

In [12]:
texts = [tokenize(document) for document in documents]

In [13]:
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

In [14]:
min_token_frequency = 10

In [15]:
texts = [[token for token in text if frequency[token] > min_token_frequency] for text in texts]

In [16]:
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

## Finding the Optimum Number of Topics

Now that the data is ready, we can run a batch LDA (because of the small size of the dataset that we are working with) to discover the main topics in our document.

In [17]:
# fit LDA model
feedback_topics = LdaModel(corpus=corpus,
                           id2word=dictionary,
                           num_topics=10,
                           passes=3)

In [18]:
# print out first 10 topics
for i, topic in enumerate(feedback_topics.print_topics(10)):
    print(i, topic)

0 (0, '0.032*"mehr" + 0.032*"velo" + 0.023*"bessere" + 0.021*"züge" + 0.021*"velos" + 0.020*"tragen" + 0.016*"kinderwagen" + 0.015*"platz" + 0.012*"nein" + 0.012*"neuen"')
1 (1, '0.061*"zug" + 0.018*"zürich" + 0.017*"bus" + 0.017*"verspätung" + 0.017*"ab" + 0.017*"minuten" + 0.015*"bern" + 0.013*"anschluss" + 0.013*"olten" + 0.012*"umsteigen"')
2 (2, '0.049*"app" + 0.037*"sbb" + 0.022*"ticket" + 0.015*"lösen" + 0.012*"billet" + 0.011*"billett" + 0.011*"konnte" + 0.011*"kaufen" + 0.010*"wurde" + 0.010*"gelöst"')
3 (3, '0.050*"bahnhof" + 0.022*"zug" + 0.018*"beim" + 0.013*"perron" + 0.012*"gleis" + 0.010*"aussteigen" + 0.009*"durchsagen" + 0.007*"wäre" + 0.007*"menschen" + 0.007*"einsteigen"')
4 (4, '0.019*"sbb" + 0.018*"wlan" + 0.016*"kontrolle" + 0.012*"steckdosen" + 0.011*"mal" + 0.011*"ja" + 0.010*"chf" + 0.008*"immer" + 0.008*"funktioniert" + 0.007*"abo"')
5 (5, '0.034*"sbb" + 0.023*"ga" + 0.022*"danke" + 0.021*"öv" + 0.020*"teuer" + 0.019*"preise" + 0.019*"finde" + 0.019*"zufrieden

In [19]:
vis_data = gensimvis.prepare(feedback_topics, corpus, dictionary)
pyLDAvis.display(vis_data)

/usr/local/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/usr/local/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is dep